In [ ]:
import torch
device = torch.device('cuda')
device

device(type='cuda')

# 1.  데이터 준비

In [ ]:
pip install datasets

In [ ]:
import json
data = []
with open('finalDataaa.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns=['ko', 'de'])

In [ ]:
df.head()

,ko,de
0,그렇게하자~ 나도 구경해보고 싶어.,Machen wir das ~ Ich möchte auch einen Blick d...
1,해변을 보고 나서 저녁 먹을 식당도 미리 알아보자!,"Nachdem wir den Strand gesehen haben, suchen w..."
2,둘러봐야지 뭘살지 알지~,"Du musst dich umschauen, so dass du weißt was ..."
3,그러면 내가 오늘 저녁 살게.,Dann werde ich dich heute zum Abendessen einla...
4,혹시 이 제품 중 오늘 들어온 도시락은 어떤걸까요?,"Was ist mit der Lunchbox, die heute hereinkam?"


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'de'],
        num_rows: 9744
    })
    test: Dataset({
        features: ['ko', 'de'],
        num_rows: 1083
    })
})

In [ ]:
dataset["validation"] = dataset.pop("test")

In [ ]:
dataset["train"][0]

{'ko': '피망과 캡시컴의 차이는 뭔가요',
 'de': 'was ist der unterschied zwischen paprika und spanischem pfeffer'}

# 2. 데이터 처리

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-ko-de'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [ ]:
def preprocess_function(examples):
    inputs = [ex for ex in examples['ko']]
    targets = [ex for ex in examples['de']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names,)

Map:   0%|          | 0/9744 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1083 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9744
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1083
    })
})

# 3. Trainer API로 모델 미세 조정하기

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
pip install sacrebleu

In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-16-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    # 예측값/실제레이블 분리
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우
    if isinstance(preds, tuple):
        preds = preds[0]

    # 사람이 읽을 수 있게 변환 (패딩 토큰, 시작 토큰, 종료 토큰 등을 제거)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # 패딩토큰이면(레이블 없는 토큰) token_id로 치환
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리 (공백 제거,  metric.compute 입력 형식 맞추기)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pip install accelerate -U

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"my-Helsinki-NLP/opus-mt-ko-de",
    evaluation_strategy="no", # 시간 오래걸림..
    save_strategy="epoch", # 매 epoch마다 저장
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3, # 공간 이슈
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,

    push_to_hub=True, # 모델 업로드
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 4. 학습

In [ ]:
trainer.train()

Step,Training Loss
500,2.464200
1000,2.237300
1500,1.992400


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62140]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62140]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=1827, training_loss=2.1735847861504514, metrics={'train_runtime': 194.7324, 'train_samples_per_second': 150.114, 'train_steps_per_second': 9.382, 'total_flos': 166801141923840.0, 'train_loss': 2.1735847861504514, 'epoch': 3.0})

In [ ]:
max_length = 128
trainer.evaluate(max_length=max_length)

{'eval_loss': 2.2745885848999023,
 'eval_bleu': 18.554017486983412,
 'eval_runtime': 42.7983,
 'eval_samples_per_second': 25.305,
 'eval_steps_per_second': 0.794,
 'epoch': 3.0}

In [1]:
with open('채식주의자_1.txt', 'r', encoding='utf-8') as f:
    korean_data = f.readlines()

In [ ]:
from transformers import pipeline
translator = pipeline('translation_ko_to_de', model='my-Helsinki-NLP/opus-mt-ko-de')
translated_data = [translator(text)[0]['translation_text'] for text in korean_data]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
for ko, de in zip(korean_data[:10], translated_data[:10]):
    print(f'한국어: {ko.strip()}')
    print(f'독일어: {de}')
    print()

In [ ]:
import json

with open('veg_1_marian.json', 'w', encoding='utf-8') as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=4)

In [3]:
import json

with open('veg_1_marian.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

In [4]:
for ko, de in zip(korean_data[:10], loaded_data[:10]):
    print(f'한국어: {ko.strip()}')
    print(f'독일어: {de}')
    print()

한국어: 아내가 채식을 시작하기 전까지 나는 그녀가 특별한 사람이라고 생각한 적이 없었다.
독일어: Ich habe nie gedacht, dass sie etwas Besonderes war, bevor meine Frau angefangen hat zu essen.

한국어: 솔직히 말하자면, 아내를 처음 만났을 때 끌리지도 않았다.
독일어: Als ich meine Frau zum ersten Mal traf, fühlte ich mich nicht zu ihr hingezogen.

한국어: 크지도 작지도 않은 키, 길지도 짧지도 않은 단발머리, 각질이 일어난 노르스름한 피부, 외꺼풀 눈에 약간 튀어나온 광대뼈, 개성있어 보이는 것을 두려워하는 듯한 무채색의 옷차림.
독일어: Größe nicht groß und nicht kurz, lange, knauser Kopf, gelbe Haut, die sich entwickelt hat, ein bisschen gespreizt auf den Augen der Außenlider, farbende Kleidung, die aussieht, als ob sie eine personelle Erscheinung fürchten würde.

한국어: 가장 단순한 디자인의 검은 구두를 신고 그녀는 내가 기다리는 테이블로 다가왔다.
독일어: Mit den einfachsten Schuhen des Designs kam sie auf den Tisch, auf den ich gewartet hatte.

한국어: 빠르지도, 느리지도, 힘있지도, 가냘프지도 않은 걸음걸이로.
독일어: schnell, langsam, stark, aber nicht wackelig.

한국어: 내가 그녀와 결혼한 것은, 그녀에게 특별한 매력이 없는 것과 같이 특별한 단점도 없어 보였기 때문이었다.
독일어: Ich habe sie geheiratet, weil ich dachte, sie habe keine speziellen Nacht